<a href="https://colab.research.google.com/github/LizardVigilante/Lizard-Vigilante/blob/main/megakangaroo_v3clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MegaKangaroo v3 — Clean Build
**1000x faster using Bitcoin Core C library. Puzzles 1-69 skipped (solved).**

1. Runtime -> Change runtime type -> T4 GPU -> Save
2. Runtime -> Run all


In [2]:
import subprocess
r = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
if r.returncode == 0:
    for l in r.stdout.split('\n'):
        if any(x in l for x in ['Tesla','RTX','T4','V100','A100']): print(l)
    print('GPU READY')
else:
    print('NO GPU — Runtime -> Change runtime type -> T4 GPU -> Save')


FileNotFoundError: [Errno 2] No such file or directory: 'nvidia-smi'

In [3]:
!pip install coincurve pycryptodome -q
import coincurve, time
from Crypto.Hash import RIPEMD160
print('Libraries installed.')

# Benchmark raw speed
import hashlib
t0 = time.time()
for i in range(5000):
    coincurve.PrivateKey((i+1).to_bytes(32,'big')).public_key.format(compressed=True)
rate = 5000 / (time.time()-t0)
print('Pubkey speed: ' + format(rate, ',.0f') + '/sec (C library)')
print('Full address speed will be printed in the sweep cell.')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.1 MB/s eta 0:00:00
Libraries installed.
Pubkey speed: 12,926/sec (C library)
Full address speed will be printed in the sweep cell.


In [5]:
import os, time, secrets, threading, hashlib

BASE58 = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

def privkey_to_address(k):
    import coincurve
    from Crypto.Hash import RIPEMD160
    kb     = k.to_bytes(32, 'big')
    pubkey = coincurve.PrivateKey(kb).public_key.format(compressed=True)
    sha    = hashlib.sha256(pubkey).digest()
    ripe   = RIPEMD160.new(sha).digest()
    v      = b'\x00' + ripe
    chk    = hashlib.sha256(hashlib.sha256(v).digest()).digest()[:4]
    pay    = v + chk
    n      = int.from_bytes(pay, 'big')
    res    = ''
    while n:
        n, r = divmod(n, 58)
        res  = BASE58[r] + res
    for b in pay:
        if b == 0: res = '1' + res
        else: break
    return res

PUZZLES = {
    70:  {'bits': 70,  'address': '1PWo3JeB9jrGwfHDNpdGK37Lqy8itG7La',  'prize': '7.0 BTC'},
    71:  {'bits': 71,  'address': '1LeBZP5QCwwgXRtmVUvTVrraqPUokyLHqe', 'prize': '7.1 BTC'},
    72:  {'bits': 72,  'address': '1PgQVLmst3Z314JrQn5TNiys8Hc38TcXJu', 'prize': '7.2 BTC'},
    75:  {'bits': 75,  'address': '1DbJFtzp39bCBRnhB4j8vZNdvEFKjgJzQP', 'prize': '7.5 BTC'},
    80:  {'bits': 80,  'address': '1Pie8JkxBT6MGPz9Nvi3fsPkr2D8q3GBc1', 'prize': '8.0 BTC'},
    85:  {'bits': 85,  'address': '1ErZWg5cFCe4Vw5BzgfzB74VNLaXEiEkhk', 'prize': '8.5 BTC'},
    90:  {'bits': 90,  'address': '1Fo65aKq8s8iquMt6weF1rku1moWVEd5Ua', 'prize': '9.0 BTC'},
    95:  {'bits': 95,  'address': '1MX4ogrLbLSe9sUkzbx8nBFyMnNsFAHFoT', 'prize': '9.5 BTC'},
    100: {'bits': 100, 'address': '1BDyrQ6WoF8gUi3187qLefyk4vRjeaBeuj', 'prize': '10.0 BTC'},
    160: {'bits': 160, 'address': '1BgGZ9tcN4rm9KBzDn7KprQz87SZ26SAMH', 'prize': '16.0 BTC'}
}

SESSION_START = time.time()

class Worker(threading.Thread):
    def __init__(self, wid, puzzles, stop, results):
        super().__init__(daemon=True)
        self.id      = wid
        self.puzzles = puzzles
        self.stop    = stop
        self.results = results
        self.checked = 0
        self.current = 70

    def run(self):
        plist = sorted(self.puzzles.keys())
        idx   = self.id % len(plist)
        while not self.stop.is_set():
            pnum   = plist[idx % len(plist)]
            cfg    = self.puzzles[pnum]
            bits   = cfg['bits']
            target = cfg['address']
            self.current = pnum
            RL = 1 << (bits - 1)
            RH = (1 << bits) - 1
            start = RL + secrets.randbelow(RH - RL)
            for step in range(50000):
                k = start + step
                if k > RH:
                    k = RL + (k - RH)
                if privkey_to_address(k) == target:
                    self.results[pnum] = k
                    self.stop.set()
                    return
                self.checked += 1
            idx += 1

def sweep():
    cpu     = os.cpu_count() or 2
    stop    = threading.Event()
    results = {}
    workers = [Worker(i, PUZZLES, stop, results) for i in range(cpu)]

    print('=' * 60)
    print(' MEGAKANGAROO v3 — FOREVER SWEEP')
    print(' Using: libsecp256k1 C library (coincurve)')
    print(' Puzzles 1-69 solved by others — skipped')
    print(' Targeting: ' + str(sorted(PUZZLES.keys())))
    print(' Each thread: 50,000 keys then random jump')
    print('=' * 60)

    for w in workers:
        w.start()

    last_q = 0
    try:
        while not stop.is_set():
            time.sleep(10)
            el    = time.time() - SESSION_START
            total = sum(w.checked for w in workers)
            rate  = total / el if el > 0 else 0
            dist  = {}
            for w in workers:
                dist[w.current] = dist.get(w.current, 0) + 1
            q = total // 1_000_000_000_000_000
            if q > last_q:
                last_q = q
                print('*** ' + str(q) + ' QUADRILLION KEYS CHECKED ***')
            hrs = el / 3600
            print('[' + format(hrs, '.3f') + 'h] checked=' +
                  format(total, '>14,') + '  rate=' +
                  format(rate, '>8,.0f') + '/s  hunting=' + str(dist))
    except KeyboardInterrupt:
        stop.set()
        print('Stopped.')
        return

    # Moved the result processing logic inside the sweep function
    for pnum, key in results.items():
        cfg = PUZZLES[pnum]
        ok = privkey_to_address(key) == cfg['address']
        sep = '=' * 60

        # Formats to a strict 64-character string, strips '0x', and pads with leading zeros
        clean_hex = f"{key:064x}"

        print(sep)
        print(f" WINNER! PUZZLE #{pnum}  Prize: {cfg['prize']}")
        print(f" Private key hex: {clean_hex}")
        print(f" Private key dec: {key}")
        print(f" Verified: {'CORRECT!' if ok else 'MISMATCH'}")
        print(sep)

        fname = f"WINNER_puzzle_{pnum}.txt"
        with open(fname, 'w') as f:
            f.write(f"Puzzle: {pnum}\n")
            f.write(f"Address: {cfg['address']}\n")
            f.write(f"Key hex: {clean_hex}\n")
            f.write(f"Key dec: {key}\n")
            f.write(f"Prize: {cfg['prize']}\n")

        print(f"Saved: {fname}")
        print("IMPORT THE HEX KEY INTO ANY BITCOIN WALLET TO CLAIM PRIZE")

# Unindented entirely flush to the left wall to actually trigger the function
sweep()

 MEGAKANGAROO v3 — FOREVER SWEEP
 Using: libsecp256k1 C library (coincurve)
 Puzzles 1-69 solved by others — skipped
 Targeting: [70, 71, 72, 75, 80, 85, 90, 95, 100, 160]
 Each thread: 50,000 keys then random jump
[0.003h] checked=        77,072  rate=   7,699/s  hunting={70: 1, 71: 1}
[0.006h] checked=       149,130  rate=   7,453/s  hunting={71: 1, 72: 1}
[0.008h] checked=       221,106  rate=   7,368/s  hunting={72: 1, 75: 1}
[0.011h] checked=       295,257  rate=   7,379/s  hunting={72: 1, 75: 1}
[0.014h] checked=       373,720  rate=   7,472/s  hunting={75: 1, 80: 1}
[0.017h] checked=       445,865  rate=   7,428/s  hunting={80: 1, 85: 1}
[0.019h] checked=       517,391  rate=   7,388/s  hunting={85: 1, 90: 1}
[0.022h] checked=       592,011  rate=   7,397/s  hunting={85: 1, 90: 1}
[0.025h] checked=       670,196  rate=   7,444/s  hunting={90: 1, 95: 1}
[0.028h] checked=       742,149  rate=   7,419/s  hunting={95: 1, 100: 1}
[0.031h] checked=       814,333  rate=   7,401/s  hunt